In [3]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable

import torch.nn.functional as F

from graphsage.encoders import Encoder
from graphsage.aggregators import MeanAggregator
import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict


from graphsage.encoders import Encoder
from graphsage.aggregators import MeanAggregator

In [4]:
import matplotlib.pyplot as plt

In [5]:
"""
Simple supervised GraphSAGE model as well as examples running the model
on the Cora and Pubmed datasets.
"""
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())

def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes,1), dtype=np.int64)
    node_map = {}
    label_map = {}
    with open("cora/cora.content") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            feat_data[i,:] = [float(info[i]) for i in range(1,len(info)-1)]
            node_map[info[0]] = i
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]

    adj_lists = defaultdict(set)
    with open("cora/cora.cites") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists, node_map

def run_cora():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 2708
    feat_data, labels, adj_lists = load_cora()
    features = nn.Embedding(2708, 1433)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
   # features.cuda()

    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 5
    enc2.num_samples = 5

    graphsage = SupervisedGraphSage(7, enc2)
    #graphsage.cuda()
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    times = []
    for batch in range(100):
        batch_nodes = train[:256]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, 
                Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time-start_time)
        print (batch, loss.data[0])

    val_output = graphsage.forward(val) 
    print ("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print ("Average batch time:", np.mean(times))

def load_pubmed():
    #hardcoded for simplicity...
    num_nodes = 19717
    num_feats = 500
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 1), dtype=np.int64)
    node_map = {}
    with open("pubmed-data/Pubmed-Diabetes.NODE.paper.tab") as fp:
        fp.readline()
        feat_map = {entry.split(":")[1]:i-1 for i,entry in enumerate(fp.readline().split("\t"))}
        for i, line in enumerate(fp):
            info = line.split("\t")
            node_map[info[0]] = i
            labels[i] = int(info[1].split("=")[1])-1
            for word_info in info[2:-1]:
                word_info = word_info.split("=")
                feat_data[i][feat_map[word_info[0]]] = float(word_info[1])
    adj_lists = defaultdict(set)
    with open("pubmed-data/Pubmed-Diabetes.DIRECTED.cites.tab") as fp:
        fp.readline()
        fp.readline()
        for line in fp:
            info = line.strip().split("\t")
            paper1 = node_map[info[1].split(":")[1]]
            paper2 = node_map[info[-1].split(":")[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists

def run_pubmed():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 19717
    feat_data, labels, adj_lists = load_pubmed()
    features = nn.Embedding(19717, 500)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
   # features.cuda()

    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 500, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 10
    enc2.num_samples = 25

    graphsage = SupervisedGraphSage(3, enc2)
    #graphsage.cuda()
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    times = []
    for batch in range(200):
        batch_nodes = train[:1024]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, 
                Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time-start_time)
        print (batch, loss.data[0])

    val_output = graphsage.forward(val) 
    print ("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print ("Average batch time:", np.mean(times))

In [6]:
feat_data, labels, adj_lists, nodes_map = load_cora()

## 数据准备——所有节点与163号节点

In [7]:
node_list = []
source_list = []
target_list = []
for node in nodes_map:
    node_list.append(nodes_map[node])
    for end in adj_lists[nodes_map[node]]:
        source_list.append(nodes_map[node])
        target_list.append(end)
data_nolink = []
for node in nodes_map:
    for in_node in node_list:
        if in_node not in adj_lists[nodes_map[node]]:
            data_nolink.append([nodes_map[node], in_node])
data_link = np.array([source_list,target_list]).T            
data_nolink = np.array(data_nolink)
rand_indices = np.random.permutation(len(data_nolink))
all_sample = np.vstack((data_link,data_nolink[rand_indices[:len(data_link)]]))

#len(all_sample)

linkprediction_label = []
for i in range(len(all_sample)):
    if i < len(data_link):
        linkprediction_label.append([1])
    else:
        linkprediction_label.append([0])


linkprediction_label = np.array(linkprediction_label)
linkprediction_label_bcel = torch.FloatTensor(linkprediction_label)

#all_sample

In [8]:
import copy

In [9]:
adj_163 = list(adj_lists[163])
no_adj_163 = []
for node in node_list:
    if node not in adj_163:
        no_adj_163.append(node)
print(len(no_adj_163))

rand_index = np.random.permutation(len(no_adj_163))
sample_no_adj = rand_index[:len(adj_163)]
sample_index = copy.copy(adj_163)
sample_index.extend(sample_no_adj)
sample_rand_index = np.array([sample_index[i] for i in np.random.permutation(len(sample_index))])

labels_163 = []
for node in node_list :
    if node in adj_163:
        labels_163.append(1)
    else:
        labels_163.append(0)
        
all_sample_163 = copy.copy(adj_163)
all_sample_163.extend(no_adj_163)

node_prediction_163 = []
for la in labels_163:
    node_prediction_163.append([la])

node_prediction_163 = np.array(node_prediction_163)

2540


In [10]:
node_prediction_163_bcel = torch.FloatTensor(node_prediction_163)

## 实验一： 163号节点连边的预测

In [11]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.BCELoss()

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes1):
        embeds1 = self.enc(nodes1)
        #embeds2 = self.enc(nodes2)
        print(embeds1.shape)
        #print(embeds2.shape)
        print(self.weight.shape)
        #print ((embeds1*embeds2).shape)#inner directly
        scores = self.weight.mm(embeds1)
        #print(scores)
        
        return scores.t()

    def loss(self, nodes1, labels):
        scores = self.forward(nodes1)
        scores = F.sigmoid(scores)
        return self.xent(scores, labels)

In [ ]:
np.random.seed(1)
random.seed(1)
num_nodes = 2708
feat_data, labels, adj_lists,_ = load_cora()
features = nn.Embedding(2708, 1433)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
# features.cuda()

agg1 = MeanAggregator(features, cuda=True)
enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 1433, adj_lists, agg2,
        base_model=enc1, gcn=True, cuda=False)
enc1.num_samples = 5
enc2.num_samples = 5
#这里改一下输出的维度
graphsage = SupervisedGraphSage(1, enc2)
#graphsage.cuda()
#rand_indices = np.random.permutation(len(node_list))
test = sample_rand_index[:50]
val = sample_rand_index[50:100]
train = list(sample_rand_index[100:])

optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.5)
times = []
Loss = []
for batch in range(200):
    #batch_nodes1 = np.ones(100)*163#No.163
    batch_nodes2 = train[:40]
    batch_idx = copy.copy(train[:40])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss(batch_nodes2,
            Variable(node_prediction_163_bcel[np.array(batch_idx)]))
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time-start_time)
    #print (batch, loss.data[0])
    Loss.append(loss.data[0])
val_output = graphsage.forward(val) 
#print ("Validation F1:", f1_score(node_prediction_163_bcel[val], val_output.data.numpy().argmax(axis=1), average="micro"))
#print ("Average batch time:", np.mean(times))

In [ ]:
right = []
wrong = []
predict_list = F.sigmoid(val_output)
for i in range(len(F.sigmoid(val_output))):
    predict = predict_list[i]
    target = node_prediction_163_bcel[val][i]
    print(abs(predict - target))
    if abs(predict - target)<0.5:
        right.append(i)
    else:
        wrong.append(i)
print(len(right)/(len(right)+len(wrong)))

## 实验二 节点对的连边预测

In [20]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.BCELoss()
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes1, nodes2):
        embeds1 = self.enc(nodes1)
        embeds2 = self.enc(nodes2)
        print(embeds1.shape)
        #print(embeds2.shape)
        print(self.weight.shape)
        #print ((embeds1*embeds2).shape)#inner directly
        scores = (torch.ones((1,self.enc.embed_dim))).mm(embeds1*embeds2)
        #print(scores)
        
        return scores.t()

    def loss(self, nodes1, nodes2, labels):
        scores = self.forward(nodes1,nodes2)
        scores = F.sigmoid(scores)
        return self.xent(scores, labels)

In [ ]:
np.random.seed(1)
random.seed(1)
num_nodes = 2708
feat_data, labels, adj_lists,_ = load_cora()
features = nn.Embedding(2708, 1433)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
# features.cuda()

agg1 = MeanAggregator(features, cuda=True)
enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
        base_model=enc1, gcn=True, cuda=False)
enc1.num_samples = 5
enc2.num_samples = 5

graphsage = SupervisedGraphSage(1, enc2)
#graphsage.cuda()
rand_indices = np.random.permutation(len(all_sample))
test = rand_indices[:1000]
val = rand_indices[1000:1500]
train = list(rand_indices[1500:])

optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.01)
times = []
Loss = []
for batch in range(100):
    batch_nodes1 = all_sample[train[:32]][:,0]
    batch_nodes2 = all_sample[train[:32]][:,1]
    batch_idx = copy.copy(train[:32])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss(batch_nodes1, batch_nodes2,
            Variable(linkprediction_label_bcel[np.array(batch_idx)]))
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time-start_time)
    print (batch, loss.data[0])
    Loss.append(loss.data[0])
val_output = graphsage.forward(all_sample[val][:,0],all_sample[val][:,1]) 
print ("Validation F1:", f1_score(linkprediction_label_bcel[val], val_output.data.numpy().argmax(axis=1), average="micro"))
print ("Average batch time:", np.mean(times))

## 实验三 两个节点拼接，预测

In [27]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.BCELoss()

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim*2))
        init.xavier_uniform(self.weight)

    def forward(self, nodes1, nodes2):
        embeds1 = self.enc(nodes1)
        embeds2 = self.enc(nodes2)
        print(embeds1.shape)
        #print(embeds2.shape)
        print(self.weight.shape)
        #print ((embeds1*embeds2).shape)#inner directly
        combined = torch.cat([embeds1, embeds2], dim=0)
        scores = self.weight.mm(combined)
        #print(scores)
        
        return scores.t()

    def loss(self, nodes1, nodes2, labels):
        scores = self.forward(nodes1,nodes2)
        scores = F.sigmoid(scores)
        return self.xent(scores, labels)

In [ ]:
np.random.seed(1)
random.seed(1)
num_nodes = 2708
feat_data, labels, adj_lists,_ = load_cora()
features = nn.Embedding(2708, 1433)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
# features.cuda()

agg1 = MeanAggregator(features, cuda=True)
enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
        base_model=enc1, gcn=True, cuda=False)
enc1.num_samples = 5
enc2.num_samples = 5

graphsage = SupervisedGraphSage(1, enc2)
#graphsage.cuda()
rand_indices = np.random.permutation(len(all_sample))
test = rand_indices[:1000]
val = rand_indices[1000:1500]
train = list(rand_indices[1500:])

optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.5)
times = []
Loss = []
for batch in range(500):
    batch_nodes1 = all_sample[train[:256]][:,0]
    batch_nodes2 = all_sample[train[:256]][:,1]
    batch_idx = copy.copy(train[:256])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss(batch_nodes1, batch_nodes2,
            Variable(linkprediction_label_bcel[np.array(batch_idx)]))
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time-start_time)
    print (batch, loss.data[0])
    Loss.append(loss.data[0])
val_output = graphsage.forward(all_sample[val][:,0],all_sample[val][:,1]) 
print ("Validation F1:", f1_score(linkprediction_label_bcel[val], val_output.data.numpy().argmax(axis=1), average="micro"))
print ("Average batch time:", np.mean(times))

In [ ]:
plt.plot(Loss)
plt.xlabel('batches')
plt.ylabel('loss')
plt.show()

In [ ]:
right = []
wrong = []
predict_list = F.sigmoid(val_output)
for i in range(len(F.sigmoid(val_output))):
    predict = predict_list[i]
    target = linkprediction_label_bcel[val][i]
    print(abs(predict - target))
    if abs(predict - target)<0.5:
        right.append(i)
    else:
        wrong.append(i)
print(len(right)/(len(right)+len(wrong)))